In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()

In [3]:
data = spark.read.csv('Tree_Methods/College.csv' , inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [5]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [8]:
assembler = VectorAssembler (inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'] , outputCol='features')

In [9]:
output = assembler.transform(data)

In [10]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex')

In [11]:
output_fixed = indexer.fit(output).transform(output)

In [13]:
output_fixed.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60, features=DenseVector([1660.0, 1232.0, 721.0, 23.0, 52.0, 2885.0, 537.0, 7440.0, 3300.0, 450.0, 2200.0, 70.0, 78.0, 18.1, 12.0, 7041.0, 60.0]), PrivateIndex=0.0)]

In [16]:
final_data  = output_fixed.select ('features','PrivateIndex')

In [17]:
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

In [19]:
from pyspark.ml import Pipeline

In [33]:
dtc = DecisionTreeClassifier (labelCol='PrivateIndex' , featuresCol='features')
rfc = RandomForestClassifier (numTrees=150 ,labelCol='PrivateIndex' , featuresCol='features')
gbt = GBTClassifier (labelCol='PrivateIndex' , featuresCol='features')

In [34]:
# Fitting data to the created model
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [35]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [36]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [37]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [38]:
my_binary_eval.evaluate(dtc_preds)

0.9061899269511726

In [39]:
rfc_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [40]:
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [41]:
# gbt prediction model don't have any detault rawPrediction column
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex',rawPredictionCol='prediction')

In [42]:
my_binary_eval2.evaluate(gbt_preds)

0.8749999999999999

In [43]:
my_binary_eval.evaluate(rfc_preds)

0.9564590542099194

In [44]:
# Evaluation using multiclass classificaion evaluator (accuracy)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [48]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',metricName='accuracy')

In [51]:
rfc_acc = acc_eval.evaluate(rfc_preds)

In [52]:
rfc_acc

0.9140271493212669